In [3]:
import numpy as np
#from counting_grid import CountingGrid

In [32]:
class CountingGrid():
    def __init__(self, size, window_size, nr_of_features):
        '''
        size-- a two-dimensional numpy array, indicating the size of the
        Counting Grid
        window_size-- a two-dimensional numpy array, indicating the size
        of the window
        '''
        np.random.seed(7)
        self.size = size
        self.window_size = window_size
        self.nr_of_features = nr_of_features # No. of features
        #Initialize arrays here for pi, h
        #pi is a 3D array, storing the probability distributions
        #that collectively characterize the data. The first dimensionality
        #corresponds to features, e.g. it is Z. The second and third
        #dimension correspond to the size of the grid in x and y directions.
        rand_init_pi = 1 + np.random.rand(self.nr_of_features,self.size[0],self.size[1])        
        self.pi = rand_init_pi/sum(rand_init_pi,0)
        #Test pi    
        #self.pi = np.array([[[0.494, 0.524],[0.479,0.418]],[[0.506, 0.476],[0.521, 0.582]]])       
        self.h = np.zeros((self.nr_of_features,self.size[0],self.size[1]))    
        #self.compute_histograms()
        
    def compute_sum_in_a_window(self,grid,k,l):
        cumsum1=np.sum(np.sum(grid[:,:k+self.window_size[0],:l+self.window_size[1]],axis=1),axis=1)
        
        cumsum2=np.sum(np.sum(grid[:,:k,:l+self.window_size[1]],axis=1),axis=1)
        
        cumsum3=np.sum(np.sum(grid[:,:k+self.window_size[0],:l],axis=1),axis=1)
        cumsum4=np.sum(np.sum(grid[:,:k,:l],axis=1),axis=1)
        cumsum=cumsum1-cumsum2-cumsum3+cumsum4
        '''
        cumsum1 = grid[self.window_size[0]-1:,self.window_size[1]-1:]
        cumsum2  = grid[:grid.shape[0]-self.window_size[0]+1,self.window_size[1]-1:]
        cumsum3 = grid[self.window_size[0]-1:,:grid.shape[1]-self.window_size[1]+1]    
        cumsum4 = grid[:grid.shape[0]-self.window_size[0]+1,:grid.shape[1]-self.window_size[1]+1]    
        #print cumsum1.shape, cumsum2.shape,cumsum3.shape    
        cumsum =  cumsum1+cumsum2+cumsum3+cumsum4
        cumsum = cumsum[:cumsum.shape[1]-1,:cumsum.shape[1]-1]
        '''
        return cumsum
    
    def compute_histograms(self):
        '''
        Histograms at each point in the grid are computed
        by averaging the distributions pi in a pre-defined
        window. The left upmost corner of the window is placed
        on the grid position and the distributions are averaged.
        '''
        for k in range(0,self.size[0]):
            for l in range(0,self.size[1]):
                self.h[:,k,l]=self.compute_sum_in_a_window(self.pi,k,l)
        self.h=self.h/np.prod(self.window_size)
        print(self.h)
        
    def e_step(self,X):
        '''
        q is a 3D array with shape q.shape=(z_dimension=
        nr_of_samples,x and y=grid_size). 
        It stores the probabilities of a sample mapping to a 
        window in location k=[i1,i2]

        h is a 3D array with shape h.shape(z_dimension=
        nr_of_features, x and y=grid_size). 
        h describes the histograms (spanning along the first axis) from 
        which samples are drawn, in each location on the grid k=[i1,i2]
        '''
        nr_of_samples = X.shape[0]
        #Determine a minimal considered probability, for numerical purposes
        min_numeric_probability = float(1)/(10*self.size[0]*self.size[1])
        #Initialize q
        q_size=(nr_of_samples,self.size[0],self.size[1])
        self.q = np.zeros(q_size)
        self.q = np.exp(np.tensordot(X,np.log(self.h),axes=(1,0)))    
        self.q[self.q<min_numeric_probability]=min_numeric_probability   
        for t in range(0,nr_of_samples):
            normalizer=np.sum(self.q[t,:,:])              
            self.q[t,:,:]= self.q[t,:,:]/normalizer 
        print(self.q)
        
        
    def update_pi(self,X):  
        '''
        Updating the distributions pi on the grid involves
        calculations on data, distributions of mappings of 
        data on the grid log_q and the histograms on each
        grid point.
        '''
        #padded_q=np.lib.pad(self.q, ((0,0),(0,self.window_size[0]),(0,self.window_size[1])),'wrap')
        #padded_h=np.lib.pad(self.h, ((0,0),(0,self.window_size[0]),(0,self.window_size[1])),'wrap')   
        new_pi=np.zeros([self.nr_of_features,self.size[0],self.size[1]])     
    
        for i1 in range(0, self.size[0]):
            for i2 in range(0,self.size[1]):
                for z in range(0,X.shape[1]):
                    t_storage=[]
                    for t in range(0,X.shape[0]):
                        window_sum=self.compute_sum_in_a_window(np.divide(self.q[t,:,:],self.h[z,:,:].reshape(1,self.size[0],self.size[1])),i1,i2)
                        #print(np.divide(self.q[t,:,:],self.h).shape)
                        interm= X[t,z]*window_sum
                        t_storage.append(interm)
                    new_pi[z,i1,i2]=self.pi[z,i1,i2]*sum(t_storage)
        self.pi=new_pi
        normalizer=np.sum(self.pi,0)
        self.pi=np.divide(self.pi,normalizer)

    
                
        
    
class CountingGridTest():
    def test_h_computation(self):
        self.pi
        return 0
    def test_e_step(self):
        return 0
    
    def test_m_step(self):
        return 0
    
    def test_arr_sum(self):
        return 0

In [34]:
cg_obj=CountingGrid(np.array([5,5]),np.array([2,2]),3)
cg_obj.compute_histograms()
X=np.array([[1,2,3],[3,4,5]])
cg_obj.e_step(X)
cg_obj.update_pi(X)
print(cg_obj.pi)

[[[0.36276502 0.32504596 0.30099668 0.34584886 0.18393083]
  [0.35711285 0.33432482 0.29840199 0.3014378  0.15966794]
  [0.33329783 0.31668244 0.32518974 0.30349302 0.14223587]
  [0.35678682 0.31643476 0.32710895 0.33937656 0.15327864]
  [0.18600527 0.17766831 0.16153802 0.18086445 0.08851516]]

 [[0.33021305 0.34510168 0.34470857 0.32172688 0.15034454]
  [0.33978469 0.34563195 0.356336   0.33769106 0.15159748]
  [0.32174262 0.3284022  0.33574103 0.33506598 0.1689451 ]
  [0.3155256  0.34243307 0.32610964 0.30984831 0.17292576]
  [0.1618791  0.18581211 0.17761193 0.15112565 0.08072554]]

 [[0.30702193 0.32985236 0.35429475 0.33242426 0.16572464]
  [0.30310247 0.32004323 0.345262   0.36087114 0.18873458]
  [0.34495956 0.35491536 0.33906923 0.361441   0.18881903]
  [0.32768758 0.34113218 0.34678141 0.35077513 0.1737956 ]
  [0.15211563 0.13651958 0.16085005 0.1680099  0.08075931]]]
[[[0.04 0.04 0.04 0.04 0.04]
  [0.04 0.04 0.04 0.04 0.04]
  [0.04 0.04 0.04 0.04 0.04]
  [0.04 0.04 0.04 0.04

In [9]:
cg_obj=CountingGrid(np.array([5,5]),np.array([2,2]),3)
#X has dimensions different from the counts in 
#MATLAB implementation-- one row is one data vector,
#that's how it is in sci-kit learn
X=np.array([[1,2,3],[3,4,5]])
print(cg_obj.compute_sum_in_a_window(cg_obj.pi,4,2))
print(cg_obj.pi)
#np.sum(cg_obj.pi[0,:2,:2])
print(np.sum(cg_obj.pi[2,1:3,1:3]))

[0.64615207 0.71044772 0.64340021]
[[[0.29635248 0.37632129 0.29822722 0.35062672 0.40694111]
  [0.41083835 0.36754795 0.25808737 0.29704539 0.3287822 ]
  [0.30684227 0.34322282 0.36844113 0.27003406 0.30988956]
  [0.41534963 0.26777659 0.28728922 0.37499452 0.25905392]
  [0.31010278 0.4339183  0.27675492 0.36939715 0.35406063]]

 [[0.31200873 0.32208919 0.36291471 0.33954886 0.30196772]
  [0.32174166 0.36501264 0.33039017 0.34598054 0.29941042]
  [0.335839   0.33654546 0.35057953 0.39839378 0.30697948]
  [0.31600325 0.29858275 0.32790107 0.26608976 0.3688009 ]
  [0.33311521 0.31440118 0.42884726 0.28160045 0.32290215]]

 [[0.39163879 0.30158952 0.33885807 0.30982442 0.29109117]
  [0.26742    0.26743942 0.41152245 0.35697407 0.37180737]
  [0.35731873 0.32023171 0.28097934 0.33157215 0.38313096]
  [0.26864712 0.43364066 0.3848097  0.35891572 0.37214518]
  [0.35678201 0.25168052 0.29439782 0.3490024  0.32303722]]]
1.2801729225906024
